In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import Functions needed for Data Preprocessing
from data import ( preprocess_stock_data,
                  clean_financial_ratios,
                  merge_and_clean_data,
                  filter_useful_features,
                  calculate_technical_indicators,
                  normalize_data,
                  merge_macro_data)

# import helper functions to run LSTM Training and Predictions
from helper import (create_df_per_stock,
                    run_for_stocks,
                    get_best_configuration,
                    final_df_cleaning,
                    create_return_arrays)

# Load Dataset

In [2]:
# Load Data
stock = pd.read_csv('../data/stocks_1.csv')
stock_factor = pd.read_csv('../data/ratios_2.csv')
macro = pd.read_csv('../data/bond_and_cpi.csv')

In [3]:
# Select stocks
stock_avg = preprocess_stock_data(stock)
# Clean financial ratios data
stock_factor_1 = clean_financial_ratios(stock_avg, stock_factor)
# Merge trading data and financial ratios data and select proper companies
stock_all_final = merge_and_clean_data(stock_avg, stock_factor_1)
# Select features that are meaningful and useful
stock_final = filter_useful_features(stock_all_final)
# Calculate momentum technical indicators
stock_final_1 = stock_final.groupby('gvkey').apply(calculate_technical_indicators).reset_index(drop=True)
stock_use = stock_final_1.dropna()
# Merge macro data
stock_use = merge_macro_data(stock_use, macro)
# Normalization
stock_n = normalize_data(stock_use)
# Save to CSV
stock_n.to_csv('../data/normalized_data.csv', index=False)

In [4]:
data = pd.read_csv("../data/normalized_data.csv") # Load Dataset from File generated previously. 
data = final_df_cleaning(data) # Drop unnecessary columns and arrange data by ticker and dates
tickers = data['tic'].unique() # Create a List of the Unique Stock Tickers
data.head()

,index,datadate,tic,cshtrm,prccm,prchm,prclm,trt1m,CAPEI,evm,...,b30ret,b20ret,b10ret,b7ret,b5ret,b2ret,b1ret,t90ret,t30ret,cpiret
0,3388,2011-03-31,AMZN,0.057669,0.050441,0.047726,0.048270,0.449252,0.657229,0.104348,...,0.447572,0.505968,0.485734,0.460448,0.479048,0.409137,0.349846,0.074603,0.021886,0.009751
1,3389,2011-04-30,AMZN,0.055855,0.054863,0.052030,0.052741,0.479247,0.657454,0.104348,...,0.524495,0.625134,0.632978,0.696662,0.777043,0.612828,0.390447,0.068691,0.018939,0.006439
2,3390,2011-05-31,AMZN,0.051408,0.055111,0.054307,0.057432,0.427200,0.657571,0.104227,...,0.577259,0.700208,0.734779,0.755825,0.762814,0.561705,0.414261,0.039340,0.010522,0.004704
3,3391,2011-06-30,AMZN,0.046094,0.057310,0.054270,0.054622,0.449368,0.657700,0.104227,...,0.352766,0.407129,0.432365,0.440776,0.524773,0.489151,0.358103,0.047289,0.015152,-0.001071
4,3392,2011-07-31,AMZN,0.044648,0.062395,0.059825,0.061283,0.479955,0.657969,0.104227,...,0.624880,0.760939,0.794987,0.803837,0.839120,0.529782,0.281747,0.009376,0.001894,0.000886


In [5]:
tickers

array(['AMZN', 'AVY', 'AXON', 'BBWI', 'BKNG', 'BLDR', 'CBRE', 'CDNS',
       'CE', 'CF', 'CHD', 'CMCSA', 'CMS', 'CNC', 'COST', 'DECK', 'DLTR',
       'EA', 'EQIX', 'FI', 'FICO', 'GOOGL', 'INCY', 'LULU', 'MA', 'MOH',
       'NDAQ', 'NI', 'NVDA', 'ODFL', 'OKE', 'PKG', 'SBAC', 'STLD', 'TDG',
       'TGT', 'TYL', 'UNH', 'URI', 'V', 'VLO', 'WST'], dtype=object)

In [6]:
# Dictionary to hold dataframe for each stock
df_per_stock = create_df_per_stock(tickers=tickers, dataframe=data)

/Users/talhajamal/Library/Mobile Documents/com~apple~CloudDocs/Documents/Imperial/Courses/Semester 3/Big Data 2/BDF2/Coursework2/src/helper.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trt1m'] = df['trt1m'].shift(-1) # Shift Target Return up


# Hyperparameter Tuning for LSTMs and Return Predictions

In [8]:
# Test with small list
param_grid = {
    'lstm_units': [100, 150],
    'dense_units1': [50, 100, 150],
    'dense_units2': [50, 75],
    'batch_size': [32, 64],
    'optimizer': ['adam']
}

In [ ]:
models = run_for_stocks(tickers, df_per_stock, param_grid)

In [ ]:
# Get Best Configuration for each stock via the CSV files
best_configurations = get_best_configuration(tickers=tickers)
best_configurations_df = pd.DataFrame(best_configurations).T.reset_index()
best_configurations_df.columns = ['ticker', 'lstm_units', 'dense_units1', 'dense_units2', 'batch_size', 'optimizer', 'avg_val_mse']
best_configurations_df
best_configurations_df.to_csv("../results/best_configs.csv")

In [22]:
# Create a Numpy Array of Returns
default_lstm_no_burning_window_12m = create_return_arrays(tickers=tickers,folder="../default_lstm_no_burning_window_12m")
default_lstm_with_burning_window_12m = create_return_arrays(tickers=tickers, folder="../default_lstm_burning_window_12m")
default_lstm_no_burning_window_3m = create_return_arrays(tickers=tickers, folder='../default_lstm_no_burning_window_3m')

In [11]:
default_lstm_no_burning_window_12m.shape

(42, 29)

In [12]:
default_lstm_with_burning_window_12m.shape

(42, 19)

In [23]:
default_lstm_no_burning_window_3m.shape

(42, 30)

# Covariance Matrix via Shrinkage

In [ ]:
# Covariance Matrix

# Use below Data for Presentation Slides

In [17]:
# Lets first try to get the model to run for 1 stock
amzn = data[data['tic'] == 'AMZN'].iloc[:, :]
y = amzn['trt1m'].values
amzn.drop(columns=['trt1m'], inplace=True)
X = amzn.iloc[:, 2:].values
print(f"The Shape of X Features is: {X.shape}")
print(f"The Shape of y Features is: {y.shape}")

The Shape of X Features is: (154, 74)
The Shape of y Features is: (154,)


In [18]:
sequence_length = 3 # Feed past 12 month returns into sequence for LSTM
# Converting Features into 3D space for LSTM to add a time component
X_features, y_target = [], []
for i in range(X.shape[0] - sequence_length):
    X_features.append(X[i:i+sequence_length])
    y_target.append(y[i + sequence_length])
X_features = np.array(X_features)
y_target = np.array(y_target)

print(f"The Shape of X Features is: {X_features.shape}")
print(f"The Shape of y Features is: {y_target.shape}")

The Shape of X Features is: (151, 3, 74)
The Shape of y Features is: (151,)


In [19]:
train_size = int(len(amzn) * 0.8)
print(f"The size of my training set will be : {train_size} and the test set will be : {int(len(amzn)) - train_size}")
X_train, y_train = X_features[:train_size], y_target[:train_size]
y_train.reshape(-1, 1)
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
X_test, y_test = X_features[train_size:], y_target[train_size:]
y_target.reshape(-1, 1)
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_test: {y_test.shape}")

The size of my training set will be : 123 and the test set will be : 31
Shape of X_train: (123, 3, 74)
Shape of y_train: (123,)
Shape of X_test: (28, 3, 74)
Shape of y_test: (28,)


In [29]:
amzn.iloc[-30:, :]

,index,datadate,tic,cshtrm,prccm,prchm,prclm,CAPEI,evm,pe_op_dil,...,b30ret,b20ret,b10ret,b7ret,b5ret,b2ret,b1ret,t90ret,t30ret,cpiret
124,3512,2021-07-31,AMZN,0.040016,0.938062,1.000000,1.000000,0.658609,0.104147,0.716613,...,0.618620,0.705042,0.723923,0.683953,0.671272,0.503478,0.374207,0.032205,0.014310,0.004811
125,3513,2021-08-31,AMZN,0.029908,0.978446,0.920324,0.960308,0.658236,0.104141,0.716435,...,0.418307,0.490286,0.443054,0.422041,0.475028,0.437820,0.328352,0.032409,0.018098,0.002066
126,3514,2021-09-30,AMZN,0.029764,0.926062,0.940849,0.990021,0.658048,0.104141,0.716369,...,0.342781,0.378163,0.339313,0.336559,0.402161,0.430359,0.363630,0.042601,0.014731,0.002716
127,3515,2021-10-31,AMZN,0.030267,0.950707,0.922026,0.960456,0.658139,0.104141,0.716400,...,0.547436,0.505639,0.442476,0.368502,0.392915,0.331917,0.297578,0.019364,0.015572,0.008308
128,3516,2021-11-30,AMZN,0.036344,0.988677,0.997101,0.992913,0.657903,0.104141,0.716586,...,0.569728,0.614801,0.612481,0.549010,0.560381,0.445607,0.312863,0.041174,0.017045,0.004913
129,3517,2021-12-31,AMZN,0.030690,0.939965,0.943471,0.999068,0.657754,0.104141,0.716517,...,0.343360,0.442497,0.443907,0.450174,0.450271,0.388509,0.250836,0.042805,0.021886,0.003073
130,3518,2022-01-31,AMZN,0.036469,0.843272,0.908504,0.818528,0.657434,0.104141,0.716382,...,0.257720,0.283518,0.269581,0.244504,0.299844,0.234217,0.133060,0.019160,0.017045,0.008415
131,3519,2022-02-28,AMZN,0.040287,0.865774,0.868385,0.836562,0.656747,0.104320,0.734715,...,0.381159,0.424750,0.460847,0.445609,0.448712,0.311586,0.225520,0.050754,0.013678,0.009134
132,3520,2022-03-31,AMZN,0.038975,0.918986,0.905489,0.807763,0.656878,0.104320,0.735914,...,0.201027,0.186209,0.064392,0.009897,0.000000,0.000000,0.028659,0.084183,0.020412,0.013351
133,3521,2022-04-30,AMZN,0.034836,0.700619,0.892566,0.735485,0.656348,0.104320,0.730995,...,0.000000,0.000000,0.006545,0.061269,0.178592,0.326329,0.194336,0.059723,0.015572,0.005583


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(data['AMZN']['trt1m'], label='AMZN Standardized Returns')
plt.plot(df_per_stock['AMZN']['trt1m'][-19:], label='Returns I want to predict')
plt.title(f'Amazon Standardized Returns')
plt.xlabel('Days')
plt.ylabel('Returns')
plt.legend()
plt.show()

In [ ]:
models['AMZN'].plot_performance()